<a href="https://colab.research.google.com/github/AmanNig/Cyberbullying-Classification-NLP/blob/main/220118_AMAN_MODEL_END_TERM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re
import nltk
nltk.download('omw-1.4')
import spacy
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,sent_tokenize # tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk
from nltk.stem import PorterStemmer
#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))
# Graphs
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,classification_report,precision_score, recall_score
from mlxtend.plotting import plot_confusion_matrix
# ML Models
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from xgboost import XGBClassifier
#keras #tf
import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot,Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,LSTM, BatchNormalization, Dropout, InputLayer, ReLU
from keras.initializers import Constant
from keras.callbacks import EarlyStopping
import keras_tuner as kt
import tensorflow_addons as tfa

F1_Score_tf = tfa.metrics.F1Score(num_classes=1,average="weighted")
auc_roc_metric_ROC = keras.metrics.AUC(curve='ROC')
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def F1_Score_tf(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    auc_roc_metric_ROC = keras.metrics.AUC(curve='ROC')


#gensim w2v  #word2vec
import gensim
from gensim.models import Word2Vec,KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
data = pd.read_csv("/kaggle/input/cyberbullying-dataset/twitter_parsed_dataset.csv")
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)
data.drop(columns=['index','id','Annotation'],inplace=True)
data

In [ ]:
def count_chars(text):
    return len(text)

# count number of words
def count_words(text):
    return len(text.split())

# count number of capital characters
def count_capital_chars(text):
    count=0
    for i in text:
        if i.isupper():
            count+=1
    return count

# count number of capital words
def count_capital_words(text):
  def count_capital_words(text):
    return sum(map(str.isupper,text.split()))

# count number of punctuations
def count_punctuations(text):
    punctuations='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    d=0
    for i in punctuations:
        d = d + text.count(i)
    return d

# count number of words in quotes
def count_words_in_quotes(text):
    x = re.findall("\'.\'|\".\"", text)
    count=0
    if x is None:
        return 0
    else:
        for i in x:
            t=i[1:-1]
            count+=count_words(t)
        return count

# count number of sentences
def count_sent(text):
    return len(nltk.sent_tokenize(text))

# calculate average sentence length
def avg_sent_len(word_cnt,sent_cnt):
    return word_cnt/sent_cnt

# count number of unique words
def count_unique_words(text):
    return len(set(text.split()))

# words vs unique feature
def words_vs_unique(words,unique):
    return unique/words
    def count_htags(text):
    x = re.findall(r'(\#\w[A-Za-z0-9]*)', text)
    return len(x)

# count of mentions
def count_mentions(text):
    x = re.findall(r'(\@\w[A-Za-z0-9]*)', text)
    return len(x)

# count of stopwords
def count_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)
    def stopwords_vs_words(stopwords_cnt,text):
     return stopwords_cnt/len(word_tokenize(text))

In [ ]:
data['char_count'] = data["Text"].apply(lambda x:count_chars(str(x)))
data['word_count'] = data["Text"].apply(lambda x:count_words(str(x)))
data['sent_count'] = data["Text"].apply(lambda x:count_sent(str(x)))
data['capital_char_count'] = data["Text"].apply(lambda x:count_capital_chars(str(x)))
data['capital_word_count'] = data["Text"].apply(lambda x:count_capital_words(str(x)))
data['quoted_word_count'] = data["Text"].apply(lambda x:count_words_in_quotes(str(x)))
data['stopword_count'] = data["Text"].apply(lambda x:count_stopwords(str(x)))
data['unique_word_count'] = data["Text"].apply(lambda x:count_unique_words(str(x)))
data['htag_count'] = data["Text"].apply(lambda x:count_htags(str(x)))
data['mention_count'] = data["Text"].apply(lambda x:count_mentions(str(x)))
data['punct_count'] = data["Text"].apply(lambda x:count_punctuations(str(x)))
data['avg_wordlength']=data['char_count']/data['word_count']
data['avg_sentlength']=data['word_count']/data['sent_count']
data['unique_vs_words']=data['unique_word_count']/data['word_count']
data['stopwords_vs_words']=data['stopword_count']/data['word_count']

In [ ]:
data["Text"][145:155].values
data["Text"][2550:2560].values

In [ ]:
import pickle
with open('/kaggle/input/emoji-dictionary-1/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

with open('/kaggle/input/emotions-dictionary-for-nlp/Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)
Emoticon_Dict = {v: k for k, v in Emoticon_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

def convert_emoticons(text):
    for emot in Emoticon_Dict:
        text = re.sub(u'('+emot+')', "_".join(Emoticon_Dict[emot].replace(",","").split()), text)
    return text
    import pickle
with open('/kaggle/input/emoji-dictionary-1/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

with open('/kaggle/input/emotions-dictionary-for-nlp/Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)
Emoticon_Dict = {v: k for k, v in Emoticon_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

def convert_emoticons(text):
    for emot in Emoticon_Dict:
        text = re.sub(u'('+emot+')', "_".join(Emoticon_Dict[emot].replace(",","").split()), text)
    return text

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

column = "Text"
data[column] = data[column].progress_apply(lambda text: convert_emoticons(str(text)))
data[column] = data[column].progress_apply(lambda text: convert_emojis_to_word(str(text)))
print( data["Text"][145:155].values )
print( data["Text"][2550:2560].values )

In [ ]:
data_copy = data.copy()
data = data_copy.copy()
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

data[column] = data[column].apply(lambda text: remove_urls(str(text)))
data[column] = data[column].apply(lambda text: remove_html(str(text)))
data

In [ ]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

data[column] = data[column].progress_apply(lambda text: chat_words_conversion(str(text)))
data

In [ ]:
def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

data[column] = data[column].progress_apply(lambda text: remove_numbers(str(text)))
def make_lowercase(data,column):
    data[column] = data[column].str.lower()
    return data
data = make_lowercase(data,column='Text')

In [ ]:
from textblob import TextBlob
data["sentiment"] = data["Text"].progress_apply(lambda x: TextBlob(x).sentiment.polarity)
data

In [ ]:
no_words = set(['no',"wasn't","has't","didn't","hadn't","mightn't","needn't","won't","weren't","doesn't","mustn't","shouldn't","haven't","doesn","hadn","didn't","not","needn","aren't","isn","aren","isn't","wasn","hasn't","didn","shouldn","don't","couldn't","wouldn't","weren","hasn",""])
stop_words_n = set(stopwords.words('english'))-no_words

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words_n])

data["Text"] = data["Text"].progress_apply(lambda text: remove_stopwords(text))
def remove_AtSign_tags(data,column):
    data[column] = data[column].apply( lambda text: re.sub('\@[a-zA-Z]+','',text) )
    return data

data = remove_AtSign_tags(data,column='Text')
def remove_puntuations(data,column):
    data[column] = data[column].progress_apply(lambda text: str(text).translate(str.maketrans('','',string.punctuation)))
    return data
data = remove_puntuations(data,"Text")
def remove_single_char(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc

data[column] = data[column].progress_apply(lambda text: remove_single_char(str(text)))

In [ ]:
def remove_most_frequent_words(data,column,frequent_words_count):
    all_text = ' '.join(data[column].tolist())
    cnt = Counter(all_text.split())
    most_frequent_words = cnt.most_common(frequent_words_count)
    # print(most_frequent_words)
    data[column] = data[column].progress_apply(lambda text: " ".join(list(set(text.split()) - set(most_frequent_words))))
    return data

def remove_rare_words(data,column,rare_words_count):
    all_text = ' '.join(data[column].tolist())
    cnt = Counter(all_text.split())
    most_frequent_words = cnt.most_common()
    rare_words = most_frequent_words[len(most_frequent_words)-rare_words_count:]
    # print(rare_words)
    rare_words = set(dict(rare_words).keys())
 data[column] = data[column].progress_apply(lambda text: " ".join(list(set(text.split()) - rare_words)))
    return data

data = remove_most_frequent_words(data,column='Text',frequent_words_count=20)
data = remove_rare_words(data,column="Text",rare_words_count=1000)

In [ ]:
column = "Text"
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
data[column] = data[column].progress_apply(lambda text: lemmatize_words(text))
data

In [ ]:
data['oh_label'] = data['oh_label'].astype(int)

index = []
for i in tqdm(list(data.index)):
    count = len(data['Text'][i].split())
    if (count<=3) & (data['oh_label'][i]==0):
        index.append(i)

data.drop(index=index,inplace=True)

NameError: name 'data' is not defined

In [ ]:
data.to_csv("preprocessed_text_twitter_parsed_dataset.csv")
data = pd.read_csv("/kaggle/working/preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

In [ ]:
def Plot_Loss_Accuracy( history, epochs ):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    auc = history.history['accuracy']
    val_auc = history.history['val_accuracy']

    epochs = np.linspace(1,epochs,epochs)
    epochs

    fig = go.Figure(data=go.Scatter( x=epochs, y=loss, name='Loss' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_loss, name='Validation Lass' ) )
    fig.layout.update( xaxis_title = 'No. of Epochs',
                       yaxis_title = "Loss",
                       legend=dict( yanchor="bottom", y=0.01, xanchor="right", x=0.99),
                       width=750, height=650,
                       font=dict(size=18))
    fig.show()

    fig = go.Figure(data=go.Scatter( x=epochs, y=auc, name='Accuracy Score' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_auc, name='Validation Accuracy Score' ) )
    fig.layout.update( xaxis_title = 'No. of Epochs',
                       yaxis_title = "Accuracy Scores",
                       legend=dict( yanchor="bottom", y=0.01, xanchor="right", x=0.99),
                       width=750, height=650,
                       font=dict(size=18))
    fig.show()

In [ ]:
def Evaluation(model,X_train,X_test,y_train,y_test,hypertuning=False):

  plt.figure(  figsize=(12,6) )
 #print( "-----------------------------------------------------------------------------------------------------------")
 # print( model )
 # print( " For Train Set :  ")
  y_pred = model.predict(X_train)
  y_pred_proba = model.predict_proba(X_train)

  accuracy_train = accuracy_score( y_train, y_pred )
  precision_train = precision_score( y_train, y_pred )
  recall_train = recall_score(y_train, y_pred)
  F1_score_train = f1_score(y_train, y_pred)
  #print("F1_Score = ", F1_score_train )
  roc_auc_train = roc_auc_score(y_train, y_pred_proba[:,1])
  #print( classification_report( y_train, y_pred ) )

  #print( " For Test Set :  ")
  y_pred = model.predict(X_test)
  y_pred_proba = model.

In [ ]:
def apply_models_with_default_paramters(X_train,X_test,y_train,y_test):
  models_default = [ XGBClassifier(),
                     GaussianNB(), #MultinomialNB()
                     RandomForestClassifier(),
                     DecisionTreeClassifier(),
                     # ExtraTreeClassifier(), KNeighborsClassifier(),
                     # SVC(probability=True),
                     AdaBoostClassifier(),
                     ]


  F1_Score_train = []
  Accuracy_train = []
  Recall_train = []
  Precision_train = []
  ROC_AUC_Score_train = []


  F1_Score_test = []
  Accuracy_test = []
  Recall_test = []
  Precision_test = []
  ROC_AUC_Score_test = []

  Model_Name = []

  for model in models_default:
    # print(model)
    Model_Name.append( model )
    model.fit(X_train, y_train)

    accuracy_train, precision_train, recall_train, F1_score_train, roc_auc_train, accuracy_test, precision_test, recall_test, F1_score_test, roc_auc_test = Evaluation(model,X_train,X_test,y_train,y_test,False)

    F1_Score_train.append( F1_score_train )
    Accuracy_train.append( accuracy_train )
    Recall_train.append( recall_train )
    Precision_train.append( precision_train )
    ROC_AUC_Score_train.append( roc_auc_train )

    F1_Score_test.append( F1_score_test )
    Accuracy_test.append( accuracy_test)
      Recall_test.append( recall_test )
    Precision_test.append( precision_test )
    ROC_AUC_Score_test.append( roc_auc_test )

  results = pd.DataFrame()
  results['Model_Name'] = Model_Name

  train_test_f1_score_difference = np.subtract(F1_Score_train,F1_Score_test)  # To Check Overfitting/Underfitting

  results['F1_Score on Test Set'] = F1_Score_test
  results['Accuracy on Test Set'] = Accuracy_test
  results['Recall on Test Set'] = Recall_test
  results['Precision on Test Set'] = Precision_test
  results['ROC_AUC_Score on Test Set'] = ROC_AUC_Score_test

  results['F1_Score on Train Set'] = F1_Score_train
  results['Accuracy on Train Set'] = Accuracy_train
  results['Recall on Train Set'] = Recall_train
  results['Precision on Train Set'] = Precision_train
  results['ROC_AUC_Score on Train Set'] = ROC_AUC_Score_train

  results['Difference of F1_Score on train and test'] = train_test_f1_score_difference
 results = results.sort_values(by=['F1_Score on Test Set','Difference of F1_Score on train and test'],ascending = [False, True])

  return results


In [ ]:
fig = px.pie(values=[sum(data['oh_label']==0),sum(data['oh_label']==1)], names=['no(0)','yes(1)'] , title='Target')
fig.update_layout(width=300,height=300)
fig.show()

In [ ]:
data = pd.read_csv("/kaggle/working/preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

# # # Taking sample temporary sample of data for writting code
# from sklearn.model_selection import train_test_split
# data_bigger, data  = train_test_split(data, test_size=0.001, random_state=10, stratify=data['oh_label'])
# data.reset_index(inplace=True)
# data.drop(columns=['index'],inplace=True)
# data

X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["oh_label"]), data[['oh_label']], test_size=0.2, random_state=10, stratify=data['oh_label'])
X_train.reset_index( inplace=True, drop=True ), X_test.reset_index( inplace=True, drop=True ), y_train.reset_index( inplace=True, drop=True ), y_test.reset_index( inplace=True, drop=True )

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer( lowercase=True, ngram_range=(1, 1), max_features=300,
                               binary=False, stop_words = 'english' )

X_train_text = pd.DataFrame.sparse.from_spmatrix( vectorizer.fit_transform(X_train['Text']) )
X_test_text = pd.DataFrame.sparse.from_spmatrix( vectorizer.transform(X_test['Text']) )
# vectorizer.get_feature_names_out()

X_train = pd.concat([X_train_text,X_train.drop(columns=['Text'])], axis=1)
X_test = pd.concat([X_test_text,X_test.drop(columns=['Text'])],axis=1)
Results = apply_models_with_default_paramters(X_train,X_test,y_train,y_test)
Results

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation=tf.keras.layers.ReLU()))
model.add(Dense(256, activation=tf.keras.layers.ReLU()))
model.add(Dense(64, activation=tf.keras.layers.ReLU()))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy', F1_Score_tf])
history = model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=20, batch_size=32)

Plot_Loss_Accuracy( history, epochs=20 )

In [ ]:
data = pd.read_csv("/kaggle/working/preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data


In [ ]:
w2v_pre_model = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin',binary=True,limit=100000)
vocab = w2v_pre_model.key_to_index
# vec = w2v_pre_model['man']
# print(vec)
w2v_pre_model.most_similar('man')

In [ ]:
(data['oh_label']==0).sum(), (data['oh_label']==1).sum()

In [ ]:
def document_vector(text,model):
    # remove out-of-vocabulary words
    doc = [word for word in text if word in vocab]
    if len(doc)==0:
        mean = np.nan
    else:
        mean = np.mean(model[doc], axis=0)
    return mean


# document_vector(data['Text'][0],w2v_pre_model)
data['Text_Vec'] = data['Text'].apply( lambda text: document_vector(text.split(),w2v_pre_model))
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop(columns=["index"],inplace=True)
data.head()

In [ ]:
(data['oh_label']==0).sum(), (data['oh_label']==1).sum()

In [ ]:
df = pd.DataFrame(np.zeros(shape=(data.shape[0],300)),columns=["col_"+str(i) for i in range(300)])

for i in range(data.shape[0]):
    try:
        df.iloc[i]=data['Text_Vec'][i]# .append(data['oh_label'][i])
    except:
        df.iloc[i] = [np.nan for i in range(300)]

df['target'] = data["oh_label"]
df.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['target']),df[['target']],test_size=0.2,random_state=10,stratify=df[['target']])
X_train.shape,X_test.shape,y_train.shape,y_test.shape
Results = apply_models_with_default_paramters(X_train,X_test,y_train,y_test)
Results

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(64, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(32, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy', auc_roc_metric_ROC, F1_Score_tf])
history = model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=10, batch_size=32)

Plot_Loss_Accuracy( history, epochs=10 )

In [ ]:
data = pd.read_csv("/kaggle/working/preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

In [ ]:
embed_dim = 300
sentences_list = data['Text'].to_list()
# sentences_list[0:3]
w2v_model = gensim.models.Word2Vec(sentences=sentences_list,window=5,min_count=1,vector_size=embed_dim)
w2v_model.train(sentences_list,epochs=25,total_examples=len(sentences_list))

In [ ]:
vocab_len = w2v_model.wv.vectors.shape[0]
print("The total number of words are : ", vocab_len )
vocab = w2v_model.wv.key_to_index.keys()

In [ ]:
def document_vector(text,model):
    # remove out-of-vocabulary words
    doc = [word for word in text if word in vocab]
    if len(doc)==0:
        mean = np.nan
    else:
        mean = np.mean(model.wv[doc], axis=0)
    return mean
    data['Text_Vec'] = data['Text'].apply( lambda text: document_vector(text.split(),w2v_model))
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop(columns=["index"],inplace=True)

In [ ]:
df = pd.DataFrame(np.zeros(shape=(data.shape[0],300)),columns=["col_"+str(i) for i in range(300)])

for i in range(data.shape[0]):
    try:
        df.iloc[i]=data['Text_Vec'][i]# .append(data['oh_label'][i])
    except:
        df.iloc[i] = [np.nan for i in range(300)]

df['target'] = data["oh_label"]
df.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['target']),df['target'],test_size=0.2,random_state=10,stratify=df['target'])
X_train.shape,X_test.shape,y_train.shape,y_test.shape
Results = apply_models_with_default_paramters(X_train,X_test,y_train,y_test)
Results

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(64, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(32, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy', auc_roc_metric_ROC])
history = model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=10, batch_size=32)

Plot_Loss_Accuracy( history, epochs=10 )

In [ ]:
data = pd.read_csv("/kaggle/working/preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(data['Text'])
vocab_size = len(tok.word_index) + 1
print("Vocabulary Size: ", vocab_size )

In [ ]:
def encode_sequence_and_padding(tokenizer, data, column):
    sequences = tokenizer.texts_to_sequences(data[column]) # convert words into sequences
    max_length = int(max([len(vector) for vector in sequences ])/4)
    sequences = pad_sequences( sequences, maxlen=max_length, padding='post')
    return sequences, max_length

sequences, max_length = encode_sequence_and_padding(tok, data=data, column='Text')
sequences = pd.DataFrame(sequences)
print("Maximum length of Sentence: ",max_length)
sequences.to_csv("sequences.csv")
X_train, X_test, y_train, y_test = train_test_split(sequences, data['oh_label'], test_size=0.20, random_state = 10, stratify=data['oh_label'])

In [ ]:
model = Sequential()
model.add(Embedding( vocab_size, 300, input_length = max_length, mask_zero=True ))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',auc_roc_metric_ROC, F1_Score_tf])
model.summary()

In [ ]:
epochs=5
batch_size=32
history = model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_test,y_test))
Plot_Loss_Accuracy( history, epochs=epochs )